In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bcn-clima/2023_MeteoCat_Detall_Estacions.csv
/kaggle/input/bcn-clima/2024_MeteoCat_Detall_Estacions.csv
/kaggle/input/bcn-clima/2020_MeteoCat_Detall_Estacions.csv
/kaggle/input/bcn-clima/2022_MeteoCat_Detall_Estacions.csv
/kaggle/input/bcn-clima/2021_MeteoCat_Detall_Estacions.csv
/kaggle/input/2025-bike-availability-prediction/sample_submission_2025.csv
/kaggle/input/2025-bike-availability-prediction/metadata_sample_submission_2025.csv


## EDA

In [2]:
clima_path = '/kaggle/input/bcn-clima/'
files_csv = [os.path.join(clima_path, f)for f in os.listdir(clima_path) if f.endswith(".csv")]
print(files_csv)

['/kaggle/input/bcn-clima/2023_MeteoCat_Detall_Estacions.csv', '/kaggle/input/bcn-clima/2024_MeteoCat_Detall_Estacions.csv', '/kaggle/input/bcn-clima/2020_MeteoCat_Detall_Estacions.csv', '/kaggle/input/bcn-clima/2022_MeteoCat_Detall_Estacions.csv', '/kaggle/input/bcn-clima/2021_MeteoCat_Detall_Estacions.csv']


In [3]:
df_2023 = pd.read_csv(files_csv[0])
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18615 entries, 0 to 18614
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DATA_LECTURA  18615 non-null  object 
 1   DATA_EXTREM   10220 non-null  object 
 2   CODI_ESTACIO  18615 non-null  object 
 3   ACRÒNIM       18615 non-null  object 
 4   VALOR         18615 non-null  float64
dtypes: float64(1), object(4)
memory usage: 727.3+ KB


In [4]:
df_2023['ACRÒNIM'].value_counts()

ACRÒNIM
TM        1460
TX        1460
TN        1460
HRM       1460
HRX       1460
HRN       1460
PM        1095
PX        1095
PN        1095
PPT       1095
RS24h     1095
VVM10     1095
DVM10     1095
VVX10     1095
DVVX10    1095
Name: count, dtype: int64

Pasar el dataframe de formato largo a formato ancho:
- Mantener las variables de 'DATA_LECTURA', 'DATA_EXTREM', 'CODI_ESTACIO'
- Crear nuevas columnas a partir de los valores de 'ACRÒNIM' (nombre de las columnas) y 'VALOR' (valor)

In [5]:
df_2023_wide = df_2023.pivot(index=['DATA_LECTURA', 'DATA_EXTREM', 'CODI_ESTACIO'], columns='ACRÒNIM', values='VALOR')
df_2023_wide = df_2023_wide.reset_index()
df_2023_wide.columns.name = None

display(df_2023.head())
display(df_2023_wide.head())

,DATA_LECTURA,DATA_EXTREM,CODI_ESTACIO,ACRÒNIM,VALOR
0,2023-01-01,NaN,D5,TM,12.9
1,2023-01-01,00:23:00,D5,TX,15.2
2,2023-01-01,08:50:00,D5,TN,10.7
3,2023-01-01,NaN,D5,HRM,69.0
4,2023-01-01,09:54:00,D5,HRX,97.0


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,DATA_LECTURA,DATA_EXTREM,CODI_ESTACIO,DVM10,DVVX10,HRM,HRN,HRX,PM,PN,PPT,PX,RS24h,TM,TN,TX,VVM10,VVX10
0,2023-01-01,NaN,D5,233.0,NaN,69.0,NaN,NaN,976.9,NaN,0.0,NaN,5.8,12.9,NaN,NaN,1.9,NaN
1,2023-01-01,NaN,X2,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.8,NaN,NaN,NaN,NaN
2,2023-01-01,NaN,X4,245.0,NaN,80.0,NaN,NaN,1021.8,NaN,0.0,NaN,4.8,13.7,NaN,NaN,0.9,NaN
3,2023-01-01,NaN,X8,289.0,NaN,87.0,NaN,NaN,1015.9,NaN,0.0,NaN,4.6,11.8,NaN,NaN,0.9,NaN
4,2023-01-01,00:02:00,D5,NaN,277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5


In [6]:
df_2023_wide.describe()

,DVM10,DVVX10,HRM,HRN,HRX,PM,PN,PPT,PX,RS24h,TM,TN,TX,VVM10,VVX10
count,1095.000000,1095.000000,1460.000000,1460.000000,1460.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1460.000000,1460.000000,1460.000000,1095.000000,1095.000000
mean,211.486758,211.986301,65.854110,44.502055,86.655479,996.531689,994.366667,0.780731,998.808037,16.458174,17.970822,14.471370,22.595822,2.714155,9.410137
std,83.449662,93.022116,11.849176,12.300749,12.305226,20.606971,20.759953,3.693039,20.539709,7.312794,6.176079,6.343299,6.140696,1.387523,3.201082
min,0.000000,1.000000,24.000000,11.000000,43.000000,944.700000,940.300000,0.000000,947.600000,1.600000,1.800000,-1.400000,5.400000,0.500000,3.000000
25%,150.500000,144.500000,57.000000,36.000000,79.000000,972.100000,970.600000,0.000000,974.200000,10.000000,13.100000,9.500000,17.900000,1.800000,7.150000
50%,234.000000,219.000000,67.000000,45.000000,89.000000,1005.900000,1004.000000,0.000000,1008.300000,16.600000,17.600000,14.050000,22.500000,2.300000,8.800000
75%,272.000000,300.500000,75.000000,54.000000,97.000000,1011.950000,1010.200000,0.000000,1014.200000,23.100000,23.400000,19.900000,27.800000,3.400000,11.100000
max,354.000000,357.000000,98.000000,86.000000,100.000000,1033.700000,1032.800000,40.900000,1034.900000,30.900000,33.400000,29.500000,38.500000,11.700000,20.900000


In [7]:
# Obtener valores de la fecha
df_2023_wide['datetime'] = pd.to_datetime(df_2023_wide['DATA_LECTURA'])
df_2023_wide['year'] = df_2023_wide['datetime'].dt.year
df_2023_wide['month'] = df_2023_wide['datetime'].dt.month
df_2023_wide['day'] = df_2023_wide['datetime'].dt.day

#### agg()

In [8]:
# Calculamos la media del valor obtenido para cada tipo de metrica medida en la estación de clima por fecha

agg_dict = {medida: 'mean' for medida in df_2023['ACRÒNIM'].unique()}
df_2023_means = df_2023_wide.groupby(['year', 'month', 'day']).agg(agg_dict).reset_index()
df_2023_means.head()

,year,month,day,TM,TX,TN,HRM,HRX,HRN,PM,PX,PN,PPT,RS24h,VVM10,DVM10,VVX10,DVVX10
0,2023,1,1,12.550,15.875,9.800,80.00,98.5,61.75,1004.866667,1006.433333,1003.833333,0.000000,5.066667,1.233333,255.666667,4.633333,253.333333
1,2023,1,2,12.600,15.725,9.325,79.75,99.0,56.00,1003.866667,1006.133333,1002.433333,0.000000,4.800000,1.266667,43.666667,4.800000,116.000000
2,2023,1,3,13.125,17.200,10.000,75.00,93.5,58.75,1008.533333,1011.433333,1005.600000,0.233333,6.133333,2.333333,305.333333,7.233333,298.333333
3,2023,1,4,11.825,17.275,8.600,71.25,88.5,46.75,1011.400000,1013.066667,1010.600000,0.000000,8.800000,1.800000,267.000000,5.700000,293.666667
4,2023,1,5,11.375,16.725,7.675,62.50,78.0,37.50,1007.200000,1010.933333,1004.100000,0.000000,9.033333,1.933333,259.666667,6.500000,270.666667


## Función para preparar los datos del clima

In [9]:
def preprocess_clima(file_path):
    # leer csv
    df =  pd.read_csv(file_path)
    
    # transformar df a formato largo
    df_wide = df.pivot(index=['DATA_LECTURA', 'DATA_EXTREM', 'CODI_ESTACIO'], columns='ACRÒNIM', values='VALOR')
    df_wide = df_wide.reset_index()
    df_wide.columns.name = None

    # obtener informacion de la fecha
    df_wide['datetime'] = pd.to_datetime(df_wide['DATA_LECTURA'])
    df_wide['year'] = df_wide['datetime'].dt.year
    df_wide['month'] = df_wide['datetime'].dt.month
    df_wide['day'] = df_wide['datetime'].dt.day

    # Calculamos la media del valor obtenido para cada tipo de metrica medida en la estación de clima por fecha
    agg_dict = {medida: 'mean' for medida in df['ACRÒNIM'].unique()}
    df_means = df_wide.groupby(['year', 'month', 'day']).agg(agg_dict).reset_index()

    print(f" Archivo {file_path} procesado")
    return df_means

In [10]:
# Preprocesar los datos de clima para cada an{~}o
df_clima = pd.concat([preprocess_clima(f) for f in files_csv], ignore_index = True)
df_clima = df_clima.sort_values(['year','month','day'])
df_clima

 Archivo /kaggle/input/bcn-clima/2023_MeteoCat_Detall_Estacions.csv procesado
 Archivo /kaggle/input/bcn-clima/2024_MeteoCat_Detall_Estacions.csv procesado
 Archivo /kaggle/input/bcn-clima/2020_MeteoCat_Detall_Estacions.csv procesado
 Archivo /kaggle/input/bcn-clima/2022_MeteoCat_Detall_Estacions.csv procesado
 Archivo /kaggle/input/bcn-clima/2021_MeteoCat_Detall_Estacions.csv procesado


,year,month,day,TM,TX,TN,HRM,HRX,HRN,PM,PX,PN,PPT,RS24h,VVM10,DVM10,VVX10,DVVX10
731,2020,1,1,8.375000,12.425000,4.900000,77.250000,89.250000,59.500000,1010.333333,1011.800000,1009.400000,0.000000,8.733333,1.566667,186.333333,6.766667,300.666667
732,2020,1,2,9.025000,13.775000,5.325000,78.250000,90.000000,59.500000,1010.033333,1011.633333,1008.866667,0.000000,7.600000,2.833333,274.000000,8.166667,260.666667
733,2020,1,3,9.325000,13.000000,6.450000,71.750000,88.000000,49.750000,1009.166667,1010.700000,1007.600000,0.000000,6.766667,2.733333,289.333333,7.833333,294.333333
734,2020,1,4,9.900000,15.375000,5.850000,61.000000,82.500000,40.750000,1009.533333,1011.600000,1007.766667,0.000000,8.933333,3.066667,291.000000,12.566667,299.000000
735,2020,1,5,10.175000,15.375000,5.775000,63.500000,86.750000,34.750000,1007.033333,1008.500000,1005.600000,0.000000,9.233333,2.500000,62.666667,10.100000,233.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2024,12,27,11.066667,16.366667,7.733333,56.333333,98.000000,22.666667,1010.033333,1011.333333,1009.200000,0.000000,8.800000,1.066667,161.000000,4.400000,209.666667
727,2024,12,28,10.633333,15.800000,7.133333,56.666667,74.666667,37.666667,1008.300000,1009.633333,1007.200000,0.000000,8.900000,1.200000,239.333333,4.766667,251.666667
728,2024,12,29,10.466667,16.333333,7.000000,58.333333,79.000000,36.333333,1007.800000,1008.800000,1006.733333,0.000000,8.933333,0.766667,212.666667,3.966667,205.333333
729,2024,12,30,10.000000,15.366667,6.733333,63.333333,82.000000,40.333333,1008.666667,1009.733333,1007.766667,0.000000,8.766667,0.966667,240.333333,4.166667,225.333333


In [11]:
df_clima.to_csv('climaBCN.csv', index=False)